In [1]:
#basics
import pandas as pd
import numpy as np
import cv2 as cv
#pytorch
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
#functions file
from functions import *
#counting time
import time

from os import listdir 
from os.path import isfile, join

In [2]:
### SEED FOR RANDOM PROCESS
np.random.seed(42)
torch.manual_seed(42)

In [3]:
# note, the problematic images were removed
### PARAMETERS
super_epoch = 2         # number of iteration over ALL images
epoch = 5               # number of iteration for one image
gt_sensitivity = 0.85   # ground truth sensitivity (between 0 and 1, high value recommended)
thr_permissivity = 0.005#-> proportion of pixel kept in the image by quantile selction (foreground selection)
splitting_ratio = 0.8   # splitting ratio for train/test splitting
batch_size = 100        # batch size for NN training
learning_rate = 1e-3    # learning rate for NN training (1e-5 stabilize quite fast, let test with 1e-6)
use_weight = True       #if true, the model will take in account the imbalance between true/false values within an image
                        
save_name = 'models/new_model' # the name for the file saved at the end

In [4]:
# import the file containing the positions for each centriole (csv)
data = pd.read_csv("data/annotations.csv")

In [ ]:
# import images and the corresponding names
## Load using channels
channels = [0,1,2]
#images,names = loadAllCtrImages(channels=channels, format_="tif", path = path)

print( 'number of images ' + str(len(images)) )

In [5]:
## load using a single folder (a single type of fluorescence)
# example with CP110
path = 'data/RPE1wt_CP110+GTU88+PCNT_2/CP110/tif'

images = []
names = [f for f in listdir(path) if isfile(join(path, f))]

for n in names:
    images.append( cv.imread( path + '/' + n , cv.IMREAD_UNCHANGED) )

names = [n[:-4] for n in names]

print( 'number of images ' + str(len(images)) )

number of images 25


In [ ]:
## uncomment if you want to use a certain number of random images in the list
'''
spl_nb = 20

indexes_spl = [ i for i in range(len(images)) ]
np.random.shuffle( indexes_spl )

spl_images = []
spl_names = []

for i in indexes_spl[:spl_nb]:
    spl_images.append( images[i] )
    spl_names.append( names[i] )
    
images = spl_images
names = spl_names
'''

In [7]:
# generate the ground truth 
images_gt = groundTruthCtr( normalizeImages(images), names , data , 28 ,gt_sensitivity)

In [8]:
### OUR NEURAL NETWORK

# device setting
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# class -> you can modify it (expect input/output) but if you do so, then you'd need to use the same for predictions
class Pixel14(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(2, 21, kernel_size=2)
        self.conv2 = torch.nn.Conv2d(21, 42, kernel_size=2)
        n = 168
        self.fc1 = torch.nn.Linear(168,n,bias=True)
        self.fc2 = torch.nn.Linear(n,n,bias=True)
        self.fc3 = torch.nn.Linear(n,n,bias=True)
        self.fc4 = torch.nn.Linear(n,n,bias=True)
        self.fc5 = torch.nn.Linear(n,n,bias=True)
        # out layer
        self.fcOut = torch.nn.Linear(n, 1,bias=True)
      
    def forward(self, x):
        relu = torch.nn.functional.relu
        max_pool2d = torch.nn.functional.max_pool2d
        # convolutions
        x = relu(max_pool2d(self.conv1(x),2))
        x = relu(max_pool2d(self.conv2(x),2))
        #flatten
        x = x.view(-1,168)
        # linear layers
        x = relu(self.fc1(x))
        x = relu(self.fc2(x))
        x = relu(self.fc3(x))
        x = relu(self.fc4(x))
        x = relu(self.fc5(x))
        
        # out layer
        x = self.fcOut(x)
        return torch.sigmoid(x)

#create NN object and asscoiated items for training
model = Pixel14().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
train_complete_pixel14( images , names, data ,
                  model, optimizer,
                  super_epoch, epoch, gt_sensitivity, thr_permissivity,
                  splitting_ratio, batch_size, learning_rate, use_weight,
                  save_name)

image normalization
ground truth generation
Iterations start here


C:\Users\Grégoire\Documents\GitHub\ML_centriole_clean\functions.py:466: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  tensor_x = torch.Tensor([np.array( [i[0] , sobelization(i[0]) ]) for i in tuples_test]) # transform to torch tensor


Starting training
Epoch 0 | mean criterion: 3.34745
Epoch 1 | mean criterion: 2.42740


In [ ]:
###############################################################################

In [ ]:
#find the best threshold for predictions ! (to use it later)
t,p,f,r = findBestThr( images, names, data, model, thr_permissivity , step=0.05,  max_ = 1, min_ =0 )
print( 'the best found threshold for prediction is : ',t )
print( 'precision = ',p,' f1 = ',f,' recall ',r )